In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
from pyzotero.zotero import Zotero
import os
from sqlite3 import connect

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.dispatcher import Dispatcher
from src.db import ZoteroDatabase
from src.scihub import SciHub

In [ ]:
zot = Zotero(os.environ['ZOTERO_USER_ID'], 'user', os.environ['ZOTERO_API_KEY'], preserve_json_order=True)

z = ZoteroDatabase(local=True)
# d = Dispatcher(session=None)

In [ ]:
keys = ['XXLF2GYS', 'G3GP7TW8','UYJJPTXG','4SDR5E5R','JTWSQKIS','AG9I49JZ', '7ARSI4KK']
item_type = 'note || attachment'

# items = z.get_items(keys=keys, use_cache=False)
# items = z.get_notes(keys=keys)
items = z.get_attachments(keys=keys)
items[0]

In [ ]:
items = z.get_items(item_type='-note || attachment')
items

In [ ]:
z.get_autocomplete_tags()

In [ ]:
[d.papers.get(k,'none') for k in d.tags['none']]

In [ ]:
d.tags['none']

In [ ]:
from collections import defaultdict
tags_dict = defaultdict(list)
for i in items:
    for t in i.get('tags', [None]):
        if t is None:
            tags_dict['none'].append(i['key'])
        else: 
            tags_dict[t['tag']].append(i['key'])
dict(tags_dict)


In [ ]:
t = {}
t.setdefault('a',[])
t

In [ ]:
{t.get('tag') for i in items for t in i.get('tags', {}) if t['type'] == 0}

{t.setdefault(t['tag'],list)[t['tag']].append(i['key']) for i in items for t in i.get('tags', {})}

In [ ]:
z.get_autocompletes()

In [ ]:
z.get_autocomplete_tags()

In [ ]:
items = z.get_items()
items
dois = {i['DOI'].lower(): i['key'] for i in items if 'DOI' in i}

In [ ]:
file_types = ['pdf', 'epub', 'djvu', 'mobi', 'okular'] # alsp PDF!
# present_file_type = {a.get('path','').split('.')[-1] for a in attachments}

d = Dispatcher(session=None)
items = z.get_items(keys=[])


def check_filenames(items):
    keys = [i['key'] for i in items]
    attachments = z.get_attachments(parent_keys=keys)
    attachments_dict = {i['key']: [(a['path'], 'attachments:' + d.build_file_name(i)) for a in attachments
                                   if a.get('parentItem') == i['key']
                                   and 'path' in a 
                                   and a['path'] != 'attachments:' + d.build_file_name(i)]
                        for i in items}
    return attachments_dict


check_filenames(items)

In [ ]:
file_types = ['pdf', 'epub', 'djvu', 'mobi', 'okular'] # also PDF!

selected_keys = ['XXLF2GYS', 'G3GP7TW8','UYJJPTXG','4SDR5E5R','JTWSQKIS','AG9I49JZ', '7ARSI4KK']


import re

def get_doi(item):
    
    def get_match(s, regex):
        r = re.compile(regex, re.IGNORECASE)
        match = r.search(s)
        if match:
            return match.group()
        
    r = r'10.\d{4,9}\/[-._;()/:A-Z0-9]+'
    return item.get('DOI') or get_match(item.get('url', ''), r) or get_match(item.get('extra', ''), r)
    


items = z.get_items(keys=selected_keys)
attachments = z.get_attachments(parent_keys=selected_keys)

# attachments_dict = {i['key']: [a['path'] for a in attachments
#                                if 'path' in a and a.get('parentItem') == i['key']
#                                and (a.get('path', '').split('.')[-1]).lower() in file_types] for i in items}
# attachments_dict

attachment_keys = [a['parentItem'] for a in attachments
                   if 'parentItem' in a and (a.get('path', '').split('.')[-1]).lower() in file_types]

no_attachments_dict = {i['key']: {'url': i.get('url'), 'DOI': get_doi(i), 'ISBN': i.get('ISBN')} for i in items if i['key'] not in attachment_keys}

no_attachments_dict

# misc

In [ ]:
z.item_types()
z.item_type_fields('note')
z.item_attachment_link_modes()
# z.item_template('journalArticle')
z.item_template('attachment', 'linked_file')

# scihub

In [ ]:
import unicodedata

In [ ]:
text = 'dalsdk-.;asjdn'
text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')
text

In [ ]:
identifier = '10.1016/j.ssmph.2019.10038'

sh = SciHub(use_fallback=False)
# sh.available_base_url_list
# result = sh.fetch('http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=1648853')
result = sh.download(identifier, path='/home/boris/Downloads/paper.pdf')
# sh._get_available_scihub_urls()
result

In [ ]:
result

In [ ]:
result.keys()

In [ ]:
z = Zotero(os.environ['ZOTERO_USER_ID'], 'user', os.environ['ZOTERO_API_KEY'], preserve_json_order=True)

In [ ]:
file_name = 'test_paper.pdf'

att = z.item_template('attachment', 'linked_file')
att['title'] = file_name
att['path'] = 'attachments:' + file_name

z.check_items([att])

In [ ]:
response = z.create_items([att], parentid='AG9I49JZ')
response

In [ ]:
response['success']['0']

# lisc - retrieve DOI relations

In [ ]:
z = Zotero(os.environ['ZOTERO_USER_ID'], 'user', os.environ['ZOTERO_API_KEY'], preserve_json_order=True)
item = z.item('2BCDWE4F')['data']
# item2 = z.item('7ARSI4KK')
item['relations'] = {'dc:relation': ['http://zotero.org/users/5832834/items/TZNEQBL3',
                                     'http://zotero.org/users/5832834/items/KQNSY94T',
                                     'http://zotero.org/users/5832834/items/PIM5RGMN',
                                     'http://zotero.org/users/5832834/items/WDYQMILT']
}

# items = [{'key': 'AG9I49JZ',
#           'note': None,
#           'path': None,
#           'itemType': 'book',
#         'relations': {'dc:relation': ['http://zotero.org/users/5832834/items/7ARSI4KK']}}]
items = [item]

z.check_items(items)
# z.update_items(items)

In [ ]:
items = z.get_items(use_cache=True)
[i['data'] for i in items if i['key'] == '2MZ5V5PJ']

In [ ]:
items = [{'key': 'XXLF2GYS', 'itemType': 'journalArticle', 'title': 'Living well: a self-determination theory perspective on eudaimonia', 'url': 'https://doi.org/10.1007/s10902-006-9023-4', 'volume': '9', 'issue': '1', 'pages': '139-170', 'publicationTitle': 'Journal of Happiness Studies', 'ISSN': '1573-7780', 'date': '2008-01-01 2008-01-01', 'DOI': '10.1007/s10902-006-9023-4', 'accessDate': '2020-03-26 09:50:27', 'libraryCatalog': 'Springer Link', 'language': 'en', 'abstractNote': 'This article distinguishes between hedonic and eudaimonic approaches to wellness, with the former focusing on the outcome of happiness or pleasure and the latter focusing not so much on outcomes as on the process of living well. We present a model of eudaimonia that is based in self-determination theory, arguing that eudaimonic living can be characterized in terms of four motivational concepts: (1) pursuing intrinsic goals and values for their own sake, including personal growth, relationships, community, and health, rather than extrinsic goals and values, such as wealth, fame, image, and power; (2) behaving in autonomous, volitional, or consensual ways, rather than heteronomous or controlled ways; (3) being mindful and acting with a sense of awareness; and (4) behaving in ways that satisfy basic psychological needs for competence, relatedness, and autonomy. In fact, we theorize that the first three of these aspects of eudaimonic living have their positive effects of psychological and physical wellness because they facilitate satisfaction of these basic, universal psychological needs. Studies indicate that people high in eudaimonic living tend to behave in more prosocial ways, thus benefiting the collective as well as themselves, and that conditions both within the family and in society more generally contribute toward strengthening versus diminishing the degree to which people live eudaimonic lives.', 'shortTitle': 'Living well', 'journalAbbreviation': 'J Happiness Stud', 'dateAdded': '2020-03-26 09:50:27', 'dateModified': '2020-03-30 13:34:12', 'creators': [{'creatorType': 'author', 'firstName': 'Edward L.', 'lastName': 'Deci'}, {'creatorType': 'author', 'firstName': 'Richard M.', 'lastName': 'Ryan'}, {'creatorType': 'author', 'firstName': 'Veronika', 'lastName': 'Huta'}], 'tags': [{'tag': '# important'}, {'tag': '# read'}, {'tag': 'Well-being'}], 'collections': ['8LDK2WDV', 'TV839KDL'], 'relations': {'dc:relation': ['http://zotero.org/users/5832834/items/SDY9BMYE', 'http://zotero.org/users/5832834/items/LBPTVGZK', 'http://zotero.org/users/5832834/items/YZNSQQ9B', 'http://zotero.org/users/5832834/items/543MZJ8Z', 'http://zotero.org/users/5832834/items/9YQM53CA', 'http://zotero.org/users/5832834/items/AXWHXS9F', 'http://zotero.org/users/5832834/items/ZLD8PZ4X', 'http://zotero.org/users/5832834/items/XMKMG8D8', 'http://zotero.org/users/5832834/items/RX9HYJ5Z']}}]
# for k,v in items[0].items():
#     if v == '':
#         del items[0][k]
# del items[0]['abstractNote']
# del items[0]['dateModified']
# del items[0]['dateAdded']
items[0]['version'] = 1996

z.check_items(items)
z.update_items(items)

In [ ]:
from lisc.requester import Requester
from lisc.urls.open_citations import OpenCitations

util = 'references' # citations, references
settings = {'format': 'json'}
dois = ['10.1016/S0305-750X(01)00109-7']

urls = OpenCitations()
urls.build_url(util=util)

url = urls.get_url(util=util, segments=dois, settings=settings)
print(url)
# urls.authenticate(url)
# urls.fill_settings(format='json')
# urls.check_url(util)

req = Requester(wait_time=0.1, logging=None)
# req.check()
r = req.request_url(url)
req.close()
r.json()

# asyncio

In [ ]:
d.get_relations_by_doi('10.1016/S0305-750X(01)00109-7')

In [ ]:
import asyncio

async def get_rel(doi):
    rels = await d.get_relations_by_doi(doi)
    return rels
    
async def get_relations_async():
    loop = asyncio.get_event_loop()
    res1 = await loop.run_in_executor(None, d.get_relations_by_doi, doi)
    res2 = await loop.run_in_executor(None, d.get_relations_by_doi, doi)
#     res = await asyncio.gather(get_rel('10.1016/S0305-750X(01)00109-7'), get_rel('10.1016/S0305-750X(01)00109-7'))
    return res1, res2

asyncio.run(get_relations_async())